In [ ]:
import numpy as np
import pandas as pd
import math

from sklearn.metrics import auc, roc_curve, f1_score, recall_score, precision_score

import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras import backend as K

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - perc) + (1 - I) * perc
        # huber loss
        huber_loss = K.sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        return huber_loss + q_order_loss
    return _qloss
perc_points = [0.01, 0.25, 0.5, 0.75, 0.99]

n_steps = 3
model = tf.keras.Sequential([LSTM(64,input_shape=(n_steps, 1), activation='sigmoid',recurrent_activation='sigmoid'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='sigmoid')
])

data = pd.read_csv('yahoo3.csv')
data = data.astype('float32')
X_data = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y_true = y[n_steps:]
result = data.iloc[:, :-1]

print('ytrue',y_true.shape)
print('y',len(y))


train_threshold = math.floor(0.7 * len(data))
X_train, y_train = split_sequence(X_data[:train_threshold][y[:train_threshold] == 0], n_steps)
print('xtrain',X_train.shape)
print('ytain',y_train.shape)
X_test, y_test = split_sequence(X_data, n_steps)
print('xtest',X_test.shape)
print('ytest',y_test.shape)
model.compile(optimizer=tf.keras.optimizers.Adam(2e-3), loss=QuantileLoss(perc_points))
model.fit(X_train, y_train, epochs=50, verbose=0)
pred = model.predict(X_test)
print('pred',pred)
y_pred=list()

dataframe1 = pd.read_csv('yahoo3.csv',usecols=[0])
dataset1 = dataframe1.values
for itr in range(len(pred)):
    inner_loop=pred[itr]
    y_pred.append(np.absolute(inner_loop[0]-inner_loop[4]))

print('y_pred',y_pred)
count=0

lper = np.percentile(y_pred, 0.9)
uper = np.percentile(y_pred, 99.1)
y_pred_algo = (y_pred <= lper) | (y_pred >= uper)

predictions = np.zeros(len(y_true))
for itr in range(len(y_pred)):
    if (y_pred[itr] <= lper or y_pred[itr] >= uper):
        count = count + 1
        print('data----anomaly', dataset1[itr], 'pred', y_pred[itr])
        predictions[itr] = 1
print('count', count)

# Calculate precision, recall, F1 score, and AUC-ROC
precision = precision_score(y_true, predictions)
recall = recall_score(y_true, predictions)
f1_score = f1_score(y_true, predictions)
fpr8, tpr8, thresholds = roc_curve(y_true, predictions)
auc_roc = auc(fpr8, tpr8)

print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1_score))
print("AUC-ROC: {:.4f}".format(auc_roc))